In [16]:
import spacy
spacy.cli.download("en_core_web_sm")



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 50.7 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
import json
# Load the pre-trained English language model
nlp = spacy.load("en_core_web_sm")
# Define a dictionary that maps keywords or phrases to the expected entity types
entity_type_mapping = {
    "DATE": ["when", "what year", "what date"],
    "CARDINAL": ["how many", "how much", "what number"],
    "PERSON": ["who", "whose", "whom", "name"],
    "LOCATION": ["where", "which country", "which city", "location"],
    "ORGANIZATION": ["which company", "which organization"]
    # You can add more mappings as needed
}
# Convert entity type abbreviations to more readable words
entity_type_readable = {
    "DATE": "Date",
    "CARDINAL": "Number",
    "PERSON": "Person",
    "GPE": "geopolitical entity",
    "LOC": "location",
    "ORG": "Organization"
}
def extract_entity_types(question):
    # Create a document object using spaCy
    doc = nlp(question)
    # Store the detected entity types
    entity_types = []
    # Iterate over the entities in the document
    for ent in doc.ents:
        entity_type = ent.label_
        if entity_type not in entity_types:
            entity_types.append(entity_type)
    # Check if any keywords or phrases match the expected entity types
    question_lower = question.lower()
    for entity_type, patterns in entity_type_mapping.items():
        for pattern in patterns:
            if pattern in question_lower:
                if entity_type not in entity_types:
                    entity_types.append(entity_type)
    # If no entity types are detected, return "UNKNOWN"
    if not entity_types:
        entity_types.append("UNKNOWN")
    return entity_types
def determine_main_entity_type(question, expected_entity_types):
    # Extract entity types from the question
    extracted_entity_types = extract_entity_types(question)
    # Map the extracted types to readable forms
    entity_type_readable = {
        "ORG": "organization",
        "DATE": "date",
        "PERSON": "person",
        "LOC": "location",
        "GPE": "geopolitical entity",
        # Add more mappings as needed
    }
    readable_types = [entity_type_readable.get(entity_type, entity_type).lower() for entity_type in expected_entity_types]
    # If there's only one expected entity type, return it
    if len(readable_types) == 1:
        return readable_types[0]
    # Attempt to determine the main entity type from the extracted types
    question_lower = question.lower()
    main_entity_type = "UNKNOWN"
    max_score = 0
    for entity_type in readable_types:
        score = question_lower.count(entity_type)
        if score > max_score:
            max_score = score
            main_entity_type = entity_type
    # Enhanced heuristics for determining the main entity type
    if main_entity_type == "UNKNOWN":
        heuristics = {
            "who": "person",
            "whom": "person",
            "whose": "person",
            "where": "location",
            "when": "date",
            "what organization": "organization",
            "which organization": "organization",
            "company": "organization",
            "corporation": "organization",
            "institution": "organization",
            "city": "location",
            "country": "geopolitical entity",
            "place": "location",
        }
        for keyword, heuristic_type in heuristics.items():
            if keyword in question_lower:
                if heuristic_type in readable_types:
                    main_entity_type = heuristic_type
                    break
    # If still unknown, default to the first expected entity type as a last resort
    if main_entity_type == "UNKNOWN" and readable_types:
        main_entity_type = readable_types[0]
    return main_entity_type
# Iterate over the JSON data and extract the questions and expected answer entity types
with open("results.txt", "w", encoding="utf-8") as file:
    # Iterate over the JSON data and extract the questions and expected answer entity types
    for data in json_data:
        question = data["question"]
        entity_types = extract_entity_types(question)
        main_entity_type = determine_main_entity_type(question, entity_types)
        
        # Write the results to the TXT file
        result = f"Question: {question} | Expected answer types: {', '.join(entity_types)} | Main expected answer type: {main_entity_type}"
        file.write(result + "\n")

In [ ]:
import spacy
import json
# Load the pre-trained English language model
nlp = spacy.load("en_core_web_sm")
# Define a dictionary that maps keywords or phrases to the expected entity types
entity_type_mapping = {
    "DATE": ["when", "what year", "what date"],
    "CARDINAL": ["how many", "how much", "what number"],
    "PERSON": ["who", "whose", "whom", "name"],
    "LOCATION": ["where", "which country", "which city", "location"],
    "ORGANIZATION": ["which company", "which organization"]
    # You can add more mappings as needed
}
# Convert entity type abbreviations to more readable words
entity_type_readable = {
    "DATE": "Date",
    "CARDINAL": "Number",
    "PERSON": "Person",
    "GPE": "geopolitical entity",
    "LOC": "location",
    "ORG": "Organization"
}
def extract_entity_types(question):
    # Create a document object using spaCy
    doc = nlp(question)
    # Store the detected entity types
    entity_types = []
    # Iterate over the entities in the document
    for ent in doc.ents:
        entity_type = ent.label_
        if entity_type not in entity_types:
            entity_types.append(entity_type)
    # Check if any keywords or phrases match the expected entity types
    question_lower = question.lower()
    for entity_type, patterns in entity_type_mapping.items():
        for pattern in patterns:
            if pattern in question_lower:
                if entity_type not in entity_types:
                    entity_types.append(entity_type)
    # If no entity types are detected, return "UNKNOWN"
    if not entity_types:
        entity_types.append("UNKNOWN")
    return entity_types
def determine_main_entity_type(question, expected_entity_types):
    # Extract entity types from the question
    extracted_entity_types = extract_entity_types(question)
    # Map the extracted types to readable forms
    entity_type_readable = {
        "ORG": "organization",
        "DATE": "date",
        "PERSON": "person",
        "LOC": "location",
        "GPE": "geopolitical entity",
        "NORP": "group or affiliation",
        # Add more mappings as needed
    }
    readable_types = [entity_type_readable.get(entity_type, entity_type).lower() for entity_type in expected_entity_types]
    # If there's only one expected entity type, return it
    if len(readable_types) == 1:
        return readable_types[0]
    # Attempt to determine the main entity type from the extracted types
    question_lower = question.lower()
    main_entity_type = "UNKNOWN"
    max_score = 0
    for entity_type in readable_types:
        score = question_lower.count(entity_type)
        if score > max_score:
            max_score = score
            main_entity_type = entity_type
    # Enhanced heuristics for determining the main entity type
    if main_entity_type == "UNKNOWN":
        heuristics = {
            "who": "person",
            "whom": "person",
            "whose": "person",
            "where": "location",
            "when": "date",
            "what organization": "organization",
            "which organization": "organization",
            "company": "organization",
            "corporation": "organization",
            "institution": "organization",
            "city": "location",
            "country": "geopolitical entity",
            "place": "location",
        }
        for keyword, heuristic_type in heuristics.items():
            if keyword in question_lower:
                if heuristic_type in readable_types:
                    main_entity_type = heuristic_type
                    break
    # If still unknown, default to the first expected entity type as a last resort
    if main_entity_type == "UNKNOWN" and readable_types:
        main_entity_type = readable_types[0]
    return main_entity_type
# Iterate over the JSON data and extract the questions and expected answer entity types
with open("result1.txt", "w", encoding="utf-8") as file:
    # Iterate over the JSON data and extract the questions and expected answer entity types
    for data in json_data:
        question = data["question"]
        entity_types = extract_entity_types(question)
        main_entity_type = determine_main_entity_type(question, entity_types)
        
        # Write the results to the TXT file
        result = f" Expected answer type: {main_entity_type}"
        file.write(result + "\n")